In [24]:
import os
import json
import torch
from torchvision import transforms
from PIL import Image

def extract_features_and_targets(video_folder, json_folder):
    features = []
    positions = []
    orientations = []

    for frame_file in os.listdir(video_folder):
        if frame_file.endswith('.jpg'):  # Assuming frames are in JPEG format
            frame_path = os.path.join(video_folder, frame_file)
            json_file = os.path.splitext(frame_file)[0] + '.json'
            json_path = os.path.join(json_folder, json_file)

            # Load frame and convert to tensor
            frame = Image.open(frame_path)
            transform = transforms.Compose([transforms.Resize((224, 224)),  # Example resizing
                                            transforms.ToTensor()])
            feature = transform(frame)

            # Parse JSON file to extract position and orientation quaternion
            with open(json_path, 'r') as f:
                data = json.load(f)
                try:
                    # Extract position and orientation quaternion information from JSON
                    position = torch.tensor([data['Position']['x'], data['Position']['y'], data['Position']['z']])
                    orientation_quaternion = torch.tensor([data['Orientation_Quaternion']['x'], data['Orientation_Quaternion']['y'], data['Orientation_Quaternion']['z'], data['Orientation_Quaternion']['w']])
                except KeyError as e:
                    print(f"KeyError: '{e}' not found in JSON file:", json_file)
                    continue  # Skip this file if target information not found

            features.append(feature)
            positions.append(position)
            orientations.append(orientation_quaternion)

    # Convert lists to tensors
    features = torch.stack(features)
    positions = torch.stack(positions)
    orientations = torch.stack(orientations)

    return features, positions, orientations

# Example usage
video_folder = 'D:/slam/VSLAM/Data/TrainingDataSetGenerated_2'
json_folder = 'D:/slam/VSLAM/Data/TrainingDataSetGenerated_2'
features, positions, orientations = extract_features_and_targets(video_folder, json_folder)

# Save features, positions, and orientations as PyTorch tensors
torch.save(features, 'D:/slam/VSLAM/Extracted/Features.pt')
torch.save(positions, 'D:/slam/VSLAM/Extracted/Positions.pt')
torch.save(orientations, 'D:/slam/VSLAM/Extracted/Orientations.pt')
